In [38]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, DBSCAN
from scipy.stats import skew, kurtosis
from scipy.fft import fft
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt 

def find_optimal_eps(X):
    neighbors = NearestNeighbors(n_neighbors=5)
    neighbors_fit = neighbors.fit(X)
    distances, indices = neighbors_fit.kneighbors(X)
    distances = np.sort(distances[:, 4], axis=0)
    return distances

def plot_distances(distances):
    plt.plot(distances)
    plt.xlabel("Data Points")
    plt.ylabel("Distance to 5th Nearest Neighbor")
    plt.title("K-Distance Graph to find optimal eps")
    plt.show()

def apply_pca(X, n_components=5):
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    return X_pca

def scale_features(X):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled

def feature_extractor(data_matrix, time_interval=5):
    data_matrix_df = pd.DataFrame(data_matrix)
    feature_matrix = pd.DataFrame()
    
    feature_matrix['mean'] = data_matrix_df.mean(axis=1)
    feature_matrix['std'] = data_matrix_df.std(axis=1)
    feature_matrix['rate_of_change'] = data_matrix_df.diff(axis=1).mean(axis=1)
    feature_matrix['time_to_peak'] = data_matrix_df.idxmax(axis=1) * time_interval
    feature_matrix['auc'] = np.trapz(data_matrix, axis=1)  # NumPy method, no need to convert
    
    feature_matrix['fft_energy'] = np.abs(np.fft.fft(data_matrix, axis=1)).mean(axis=1)  # NumPy method, no need to convert
    feature_matrix['cv_glucose'] = data_matrix_df.std(axis=1) / data_matrix_df.mean(axis=1)
    feature_matrix['peak_amplitude'] = data_matrix_df.max(axis=1) - data_matrix_df.min(axis=1)
    feature_matrix['time_above_180'] = (data_matrix_df > 180).mean(axis=1)
    feature_matrix['skewness'] = data_matrix_df.apply(lambda x: skew(x), axis=1)
    feature_matrix['kurtosis'] = data_matrix_df.apply(lambda x: kurtosis(x), axis=1)
    feature_matrix['energy'] = (data_matrix ** 2).sum(axis=1)
    
    return feature_matrix

def generate_meal_feature_matrix(insulin_data, cgm_data):
    meal_times = insulin_data[insulin_data['BWZ Carb Input (grams)'].notna() & (insulin_data['BWZ Carb Input (grams)'] > 0)]
    feature_matrix = pd.DataFrame()

    for meal_time in meal_times['Timestamp']:
        cgm_meal_data = cgm_data[(cgm_data['Timestamp'] >= meal_time - pd.Timedelta(hours=2)) & (cgm_data['Timestamp'] <= meal_time + pd.Timedelta(hours=2.5))]

        if len(cgm_meal_data) >= 24:
            glucose_values = cgm_meal_data[['Sensor Glucose (mg/dL)']].values.T
            meal_features = feature_extractor(glucose_values)
            feature_matrix = pd.concat([feature_matrix, meal_features], ignore_index=True)

    return feature_matrix

def process_and_extract_features(insulin_data, cgm_data):
    insulin_data['Timestamp'] = pd.to_datetime(insulin_data['Date'] + ' ' + insulin_data['Time'])
    cgm_data['Timestamp'] = pd.to_datetime(cgm_data['Date'] + ' ' + cgm_data['Time'])

    meal_feature_matrix = generate_meal_feature_matrix(insulin_data, cgm_data)
    return meal_feature_matrix

def extract_ground_truth(insulin_data):
    carb_input = insulin_data['BWZ Carb Input (grams)'].dropna()
    min_carb = carb_input.min()
    max_carb = carb_input.max()

    bins = np.arange(min_carb, max_carb + 20, 20)
    
    ground_truth_labels = pd.cut(carb_input, bins, labels=False, include_lowest=True)
    
    return carb_input.values, ground_truth_labels.values

def cluster_data(n_bins, meal_feature_matrix):
    imputer = SimpleImputer(strategy='mean')
    meal_feature_matrix_imputed = imputer.fit_transform(meal_feature_matrix)

    scaler = StandardScaler()
    meal_feature_matrix_scaled = scaler.fit_transform(meal_feature_matrix_imputed)
    kmeans = KMeans(n_clusters=n_bins, init='k-means++', n_init=20, random_state=42)
    kmeans_labels = kmeans.fit_predict(meal_feature_matrix_scaled)

    #optimal_eps = find_optimal_eps(meal_feature_matrix_scaled)
    #plot_distances(optimal_eps)
    dbscan = DBSCAN(eps=1.5, min_samples=10)
    dbscan_labels = dbscan.fit_predict(meal_feature_matrix_scaled)

    return kmeans, kmeans_labels, dbscan_labels

def calculate_sse_for_kmeans(data, labels, cluster_centers):
    sse = 0
    for i, center in enumerate(cluster_centers):
        cluster_points = data[labels == i]
        
        if len(cluster_points) > 0:
            sse += np.sum((cluster_points - center) ** 2)
    
    return sse

def calculate_sse_for_dbscan(data, labels):
    sse = 0
    for label in np.unique(labels):
        if label != -1:  # Skip noise points
            cluster_points = data[labels == label]
            cluster_center = cluster_points.mean(axis=0)
            sse += np.sum((cluster_points - cluster_center) ** 2)
    
    return sse

def calculate_entropy(cluster_labels, ground_truth_labels, n_clusters):
    entropy = 0
    contingency_matrix = confusion_matrix(ground_truth_labels, cluster_labels)
    for i in range(n_clusters):
        cluster_size = np.sum(contingency_matrix[i])
        if cluster_size == 0:
            continue
        cluster_entropy = 0
        for j in range(len(contingency_matrix[i])):
            if contingency_matrix[i][j] > 0:
                probability = contingency_matrix[i][j] / cluster_size
                cluster_entropy -= probability * np.log2(probability)
        entropy += (cluster_size / len(ground_truth_labels)) * cluster_entropy
    return entropy

def calculate_purity(cluster_labels, ground_truth_labels, n_clusters):
    contingency_matrix = confusion_matrix(ground_truth_labels, cluster_labels)
    purity = np.sum(np.amax(contingency_matrix, axis=1)) / len(ground_truth_labels)
    return purity

def write_results_to_csv(sse_kmeans, sse_dbscan, entropy_kmeans, entropy_dbscan, purity_kmeans, purity_dbscan, output_file):
    results = [sse_kmeans, sse_dbscan, entropy_kmeans, entropy_dbscan, purity_kmeans, purity_dbscan]
    pd.DataFrame([results]).to_csv(output_file, header=None, index=False)

def align_data(meal_feature_matrix, ground_truth_labels):
    # Ensure both are numpy arrays
    meal_feature_matrix = np.asarray(meal_feature_matrix)
    ground_truth_labels = np.asarray(ground_truth_labels)
    
    # Ensure both have the same length
    min_len = min(len(meal_feature_matrix), len(ground_truth_labels))
    
    # Align both datasets
    aligned_meal_features = meal_feature_matrix[:min_len]
    aligned_ground_truth = ground_truth_labels[:min_len]
    
    # Debugging: Print sample data to inspect
    print(f"Aligned Meal Features (first row): {aligned_meal_features[0]}")
    print(f"Aligned Ground Truth Labels (first 10 labels): {aligned_ground_truth[:10]}")
    
    return aligned_meal_features, aligned_ground_truth

def main():
    insulin_file = './InsulinData.csv'
    cgm_file = './CGMData.csv'
    insulin_data = pd.read_csv(insulin_file, low_memory=False)
    cgm_data = pd.read_csv(cgm_file, low_memory=False)
    
    meal_feature_matrix_dirty = process_and_extract_features(insulin_data, cgm_data)
    meal_feature_matrix_clean = meal_feature_matrix_dirty.dropna()
    meal_feature_matrix_pca = apply_pca(meal_feature_matrix_clean, n_components=5)
    meal_feature_matrix_scaled = scale_features(meal_feature_matrix_pca)
    # Step 1: Extract ground truth labels
    carb_input, ground_truth_labels = extract_ground_truth(insulin_data)
    meal_feature_matrix, ground_truth_labels = align_data(meal_feature_matrix_scaled, ground_truth_labels)
    print(f"NaNs in feature matrix: {np.isnan(meal_feature_matrix).sum()}")
    
    # Step 2: Perform clustering (load meal feature matrix)
    n_bins = 2
    kmeans, kmeans_labels, dbscan_labels = cluster_data(n_bins, meal_feature_matrix)
    print(f"KMeans labels: {np.unique(kmeans_labels)}")
    print(f"DBSCAN labels: {np.unique(dbscan_labels)}")
    
    # Step 3: Calculate SSE
    kmeans_sse = calculate_sse_for_kmeans(meal_feature_matrix, kmeans_labels, kmeans.cluster_centers_)
    dbscan_sse = calculate_sse_for_dbscan(meal_feature_matrix, dbscan_labels) 
    print(f"KMeans SSE: {kmeans_sse}")
    print(f"DBSCAN SSE: {dbscan_sse}")
    dbscan_silhouette = silhouette_score(meal_feature_matrix, dbscan_labels)
    print(f"DBSCAN Silhouette Score: {dbscan_silhouette}")
    # Step 4: Calculate entropy and purity
    kmeans_entropy = calculate_entropy(kmeans_labels, ground_truth_labels, n_bins)
    dbscan_entropy = calculate_entropy(dbscan_labels, ground_truth_labels, n_bins)
    print(f"kmeans entropy: {kmeans_entropy}")
    print(f"DBSCAN entropy: {dbscan_entropy}")
    kmeans_purity = calculate_purity(kmeans_labels, ground_truth_labels, n_bins)
    dbscan_purity = calculate_purity(dbscan_labels, ground_truth_labels, n_bins)
    print(f"kmeans purity: {kmeans_purity}")
    print(f"DBSCAN purity: {dbscan_purity}")
    # Step 5: Output results to CSV
    write_results_to_csv(kmeans_sse, dbscan_sse, kmeans_entropy, dbscan_entropy, kmeans_purity, dbscan_purity, 'Result.csv')

main()


Aligned Meal Features (first row): [-0.39491061 -1.04644436 -0.41067801  1.40382966 -0.64378146]
Aligned Ground Truth Labels (first 10 labels): [1 0 0 3 0 0 1 0 1 0]
NaNs in feature matrix: 0
KMeans labels: [0 1]
DBSCAN labels: [-1  0]
KMeans SSE: 2086.832743031122
DBSCAN SSE: 1816.4197345635012
DBSCAN Silhouette Score: 0.4419341610011935
kmeans entropy: 0.6117155457018411
DBSCAN entropy: 0.20121056126939535
kmeans purity: 0.7585513078470825
DBSCAN purity: 0.9356136820925554
